In [1]:
# pip install langchain

In [2]:
# pip install  langchain-groq

In [3]:
from langchain_groq import ChatGroq

API_KEY = {API_Key}
llm = ChatGroq(
    model="llama-3.1-70b-versatile",
    temperature=0,
    api_key = API_KEY
)


response = llm.invoke("who is the first person on moon")
print(response.content)

The first person to walk on the Moon was Neil Armstrong, an American astronaut and the mission commander of the Apollo 11 mission. He stepped onto the lunar surface on July 20, 1969, at 2:56 UTC (Coordinated Universal Time). Armstrong famously declared, "That's one small step for man, one giant leap for mankind," as he became the first human to set foot on the Moon.

Armstrong was followed by fellow astronaut Edwin "Buzz" Aldrin, who also walked on the Moon during the mission. Michael Collins, the third member of the Apollo 11 crew, remained in orbit around the Moon in the command module.

Neil Armstrong's historic moonwalk lasted for about two and a half hours, during which he and Aldrin collected samples and conducted experiments on the lunar surface. Their mission marked a major milestone in space exploration and paved the way for future human spaceflight.


In [4]:
# pip install chromadb

In [5]:
# pip install langchain_community beautifulsoup4

In [6]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://www.google.com/about/careers/applications/jobs/results/121274223367725766-customer-engineering-manager-ii-startups-google-cloud")
page_data = loader.load().pop().page_content

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [7]:
# page_data

In [8]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
    """
    SCRAPED TEXT FROM WEBSITE:
    {page_data}
    ### INSTRUCTION :
    The scrapted text is from a job description on a website.
    Your job is to the extract the job description and return in a JSON format containing the following keys : 'role' , 'exprience' , 'skills' and 'description'
    Only return the valid json.
    And Remember NO PREAMBLE:
    
    """
)

chain_extract = prompt_extract | llm 

res = chain_extract.invoke(input={'page_data':page_data})
print(res.content)


```
{
  "role": "Customer Engineering Manager II, Startups, Google Cloud",
  "experience": "10 years of experience with cloud native architecture in a customer-facing or support role",
  "skills": [
    "Experience with software lifecycles, building tools, and architecting and developing software for scalable, distributed systems, including data platform, AI/ML and infrastructure",
    "Experience as a Pre-Sales Manager or a people manager in a technical customer-facing role within a professional services or Sales Engineering team",
    "Experience managing a team through sales processes, operations and career development, including account mapping, quota setting, quarterly/annual performance management, and managing sensitive information",
    "Experience engaging with, and presenting to, technical stakeholders and executive leaders, including tailoring and delivering compelling messages by audience, asking strategic questions, and leading conversations that drive business opportunity

In [9]:
type(res.content)

str

In [10]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)

type(json_res)

dict

In [11]:
import pandas as pd

df = pd.read_csv("./my_portfolio.csv")
df.head()

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio


In [12]:
import chromadb
import uuid 


client = chromadb.PersistentClient('store')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=['Techstack'],
                       metadatas = {'links' : row['Links']},
                       ids = [str(uuid.uuid4())])
        


In [13]:
job = json_res

In [14]:
job['skills']

['Experience with software lifecycles, building tools, and architecting and developing software for scalable, distributed systems, including data platform, AI/ML and infrastructure',
 'Experience as a Pre-Sales Manager or a people manager in a technical customer-facing role within a professional services or Sales Engineering team',
 'Experience managing a team through sales processes, operations and career development, including account mapping, quota setting, quarterly/annual performance management, and managing sensitive information',
 'Experience engaging with, and presenting to, technical stakeholders and executive leaders, including tailoring and delivering compelling messages by audience, asking strategic questions, and leading conversations that drive business opportunity']

In [15]:
links = collection.query(
    query_texts= job['skills'], # Chroma will embed this for you
    n_results=2 # how many results to return
).get('metadatas' ,[])

links

[[{'links': 'https://example.com/react-native-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/react-native-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/react-native-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/react-native-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}]]

In [16]:
promt_email = PromptTemplate.from_template(
    """
    my Job Description is {job_description}

    ## instruction
    ### INSTRUCTION:
        You are Likhit, a business development executive it is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Mohan, BDE at AtliQ. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
    """
)

chain_email = promt_email | llm 

res = chain_email.invoke({
    "job_description" : str(job),
    "link_list" : links
    
})


print(res.content)

Subject: Unlock Scalable Cloud Transformation with AtliQ

Dear Hiring Manager for Customer Engineering Manager II, Startups, Google Cloud,

I came across the job description for the Customer Engineering Manager II role at Google Cloud, and I was impressed by the emphasis on cloud native architecture, scalable distributed systems, and talent strategy. As a Business Development Executive at AtliQ, I'd like to introduce you to our AI & Software Consulting company, dedicated to facilitating seamless business process integration through automated tools.

we've empowered numerous enterprises with tailored solutions, fostering scalability, process optimization, cost reduction, and heightened overall efficiency. Our expertise in cloud native architecture, software lifecycles, and data platform development aligns perfectly with the requirements of this role. We've successfully delivered projects involving scalable, distributed systems, AI/ML, and infrastructure development.

I'd like to highlig